In [ ]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, transpile
from qiskit.circuit.library import QFT, PauliEvolutionGate
from qiskit.circuit import Parameter
from qiskit.synthesis import SuzukiTrotter
from qiskit.opflow import Z, X, Y
import random
import tqdm

from tools.classical import *
# from tools.quantum import *

#TODO: Try with Trotterized ES to see if we get close to exact result
#TODO: Use 2nd order trotter

In [ ]:
num_estimating_bits = 4
num_qubits = 2
T = 1
# For 2x2 matrix one qubit is enough
qr_sys = QuantumRegister(num_qubits, name="q")
# In QPE we use n ancillas to estimate n bits from the phase
qr_energy = QuantumRegister(num_estimating_bits, name="E") 
# For n ancillary qubit measurment we need n cllasical bits
cr_energy = ClassicalRegister(num_estimating_bits, name="c") # Create a quantum circuit
circ = QuantumCircuit(qr_energy, qr_sys, cr_energy)

In [ ]:
X_qt = qt.sigmax()
Y_qt = qt.sigmay()
Z_qt = qt.sigmaz()

coeff_lower_bound = 0
coeff_xx = np.arange(1, coeff_lower_bound, -0.1)
coeff_yy = np.arange(1, coeff_lower_bound, -0.1)
coeff_zz = np.arange(1, coeff_lower_bound, -0.1)

# get all combinations of coefficients
coeffs = np.array(np.meshgrid(coeff_xx, coeff_yy, coeff_zz)).T.reshape(-1, 3)

for coeff in coeffs:
    hamiltonian_qt = hamiltonian_matrix([X_qt, X_qt], [Y_qt, Y_qt], [Z_qt, Z_qt], coeffs=coeff, num_qubits=num_qubits)
    exact_spec = np.linalg.eigvalsh(hamiltonian_qt)
    exact_spec = np.round(exact_spec, 3)

    if len(np.unique(exact_spec)) == len(exact_spec):
        print("Unique spectrum found: ", exact_spec)
        print("Coefficients: ", coeff)
        break

coeffs = coeff / np.max(np.abs(exact_spec))
hamiltonian_qt = hamiltonian_matrix([X_qt, X_qt], [Y_qt, Y_qt], [Z_qt, Z_qt], coeffs=coeffs, num_qubits=num_qubits)
spectrum, eigenstates = np.linalg.eigh(hamiltonian_qt)
print(f'Coeffs used: {coeffs} for spectrum {spectrum}')

eigenstate = eigenstates[1]
exact_energy = spectrum[1]
print(f'Energy to predict {spectrum[1]}')


In [ ]:
def trotter_step_circ(num_qubits: int, coeffs = [1, 1, 1]):
    trotter_step_circ = QuantumCircuit(qr_sys.size)

    step_size = Parameter('theta')
    for i in range(qr_sys.size):
        if i != num_qubits - 1:
            trotter_step_circ.rxx(-2 * coeffs[0] * step_size, i, i + 1)
            trotter_step_circ.ryy(-2 * coeffs[1] * step_size, i, i + 1)
            trotter_step_circ.rzz(-2 * coeffs[2] * step_size, i, i + 1)
        if (i == num_qubits - 1):
            trotter_step_circ.rxx(-2 * coeffs[0] * step_size, i, 0)
            trotter_step_circ.ryy(-2 * coeffs[1] * step_size, i, 0)
            trotter_step_circ.rzz(-2 * coeffs[2] * step_size, i, 0)
    return trotter_step_circ

def second_order_trotter_step_circ(num_qubits: int, coeffs = [1, 1, 1]):
    qr = QuantumRegister(num_qubits, name="q")
    first_stage_circ = QuantumCircuit(qr)
    step_size = Parameter('theta')
    
    for i in range(qr.size):
        if i != num_qubits - 1:
            first_stage_circ.rxx(-2 * coeffs[0] * step_size / 2, i, i + 1)
            first_stage_circ.ryy(-2 * coeffs[1] * step_size / 2, i, i + 1)
            first_stage_circ.rzz(-2 * coeffs[2] * step_size / 2, i, i + 1)
        if (i == num_qubits - 1):
            first_stage_circ.rxx(-2 * coeffs[0] * step_size / 2, i, 0)
            first_stage_circ.ryy(-2 * coeffs[1] * step_size / 2, i, 0)
            first_stage_circ.rzz(-2 * coeffs[2] * step_size / 2, i, 0)
    
    second_stage_circ = first_stage_circ.reverse_ops()
    trotter_step_circ = first_stage_circ.compose(second_stage_circ, qr)
    
    return trotter_step_circ

def trotter_circ_for_qpe(trotter_step_circ: QuantumCircuit, total_time: float, num_trotter_steps: int) -> QuantumCircuit:
    trotter_circ = QuantumCircuit(qr_sys.size)
    step_size = 2*np.pi * total_time / num_trotter_steps
    for _ in range(num_trotter_steps):
        trotter_circ.compose(trotter_step_circ, inplace=True)
    
    trotter_circ.assign_parameters([step_size], inplace=True)

    return trotter_circ

num_trotter_steps = 200
# heisenberg_trott_circ = trotter_circ_for_qpe(second_order_trotter_step_circ(num_qubits), T, num_trotter_steps)
heisenberg_trott_circ = trotter_circ_for_qpe(trotter_step_circ(num_qubits, coeffs), T, num_trotter_steps)
cU = heisenberg_trott_circ.control(1)
# print(heisenberg_trott_circ)

In [ ]:

# #* Qiskit higher level functions doing Suzuki.. difference is only that we have periodic bdr, but otherwise both are the same.
# operator = (X^X) + (Y^Y) + (Z^Z)
# evo = PauliEvolutionGate(operator, time= -T * 2 * np.pi)
 
# # plug it into a circuit
# circuit = QuantumCircuit(2)
# circuit.append(evo, range(2))
# print(circuit.draw())
# qiskit_suzuki = SuzukiTrotter(order = 2, reps=num_trotter_steps)
# qiskit_suzuki_circ = qiskit_suzuki.synthesize(evo)
# print(qiskit_suzuki_circ.decompose())
# print(qiskit_suzuki_circ.data)

In [ ]:
initial_state = Statevector(eigenstate.reshape(2**num_qubits, 1))
circ.initialize(initial_state, qr_sys)
circ.h(qr_energy)

for n in range(qr_energy.size):
    for m in range(2**n):
        circ.compose(cU, qubits=[qr_energy[n], *list(qr_sys)], inplace=True)
        
qft_circ = QFT(num_estimating_bits, do_swaps=True, inverse=True)
circ.compose(qft_circ, qr_energy, inplace=True)
circ.measure(qr_energy, cr_energy)
tr_circuit = transpile(circ, basis_gates=['u3', 'cx'], optimization_level=3)

In [ ]:
backend = Aer.get_backend('qasm_simulator')
shots = 2**10
resulted_phase_bits = []
# for _ in tqdm.tqdm(range(20)):
job = backend.run(tr_circuit, shots=shots)
result = job.result()
counts = result.get_counts()
print(counts)
phase_bits = max(counts, key=counts.get) # take the most often obtaned result
print(f'Max count was for phase bits {phase_bits} : {counts[phase_bits]} times')

resulted_phase_bits.append(phase_bits)


In [ ]:
num_of_correct_bits = 0
for phase_bits in resulted_phase_bits:
    if phase_bits == '0100':
        num_of_correct_bits += 1
num_of_correct_bits
#* Correct Phase bits counter, 1024 shots
#* For 2nd order, 100 steps = 12/20
#* For 1st order, 100 steps = 8/20
#* For 1st order, 200 steps = 7/20

In [ ]:
phase = int(phase_bits, 2) / 2**num_estimating_bits  # exp(i 2pi phase)
    
estimated_energy = phase / T  # exp(i 2pi phase) = exp(i 2pi E T)
print(f'Exact energy: {exact_energy}')
print(f'Estimated energy: {estimated_energy}')